Download data using instagram's API

In [ ]:
import csv
#from instagram_rivate_api import Client, ClientCompatPatch
import datetime

Part 1 - data scraping
def get_instagram_posts_descriptions(username, num_posts, user, password):
   # Authenticate with the Instagram API
   api = Client(user, password)

   # Get the user ID
   user_info = api.username_info(username)
   user_id = user_info['user']['pk']

   # Get user feed
   posts = []
   results = api.user_feed(user_id)
   posts.extend(results.get('items', []))

   # Keep fetching more posts until we have enough or there are no more posts
   next_max_id = results.get('next_max_id')
   while len(posts) < num_posts and next_max_id:
       results = api.user_feed(user_id, max_id=next_max_id)
       posts.extend(results.get('items', []))
       next_max_id = results.get('next_max_id')

   # Extract descriptions and timestamps, then save to CSV
   with open(f'{username}_posts.csv', 'w', newline='', encoding='utf-8') as csvfile:
       fieldnames = ['description', 'timestamp']
       writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
       writer.writeheader()

       for post in posts[:num_posts]:
           description = post['caption']['text'] if 'caption' in post and post['caption'] else ''
           timestamp = datetime.datetime.fromtimestamp(post['taken_at']).strftime('%Y-%m-%d %H:%M:%S')
           writer.writerow({'description': description, 'timestamp': timestamp})

username = 'guardian'
num_posts = 2500
user = '' #Needs to be filled with fective instagram user
password = ''
get_instagram_posts_descriptions(username, num_posts, user, password)

Clean the data and save it into a new cleaned file

In [ ]:
import pandas as pd
import re


def clean_description(text):
    # Remove all non-alphabetical and non-numerical symbols
    cleaned_text = re.sub(r'[^A-Za-z0-9\s]', '', text)
    return cleaned_text


def clean_csv(file_name):
    # Read the input CSV file
    df = pd.read_csv(file_name)

    # Check if the 'description' column exists
    if 'description' not in df.columns:
        print(f"'description' column not found in {file_name}")
        return

    # Apply cleaning to the 'description' column
    df['cleaned_description'] = df['description'].apply(clean_description)

    # Save the results to a new CSV file
    output_file_name = 'cleaned_' + file_name.split('/')[-1]
    df.to_csv(output_file_name, index=False)
    print(f"Cleaned data saved to {output_file_name}")

